In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from googlenewsdecoder import gnewsdecoder
from urllib.parse import urlparse
from fake_useragent import UserAgent

In [4]:
import feedparser
from newspaper import Article
from urllib.parse import unquote

In [5]:
#Getting top news from google news 

def get_news(rss_url="https://news.google.com/rss?hl=en-US&gl=US&ceid=US:en"):
    response = requests.get(rss_url)
    soup = BeautifulSoup(response.content, 'xml')
    
    return [{
        'title': item.title.text,
        'source': item.source.text,
        'time': item.pubDate.text,
        'link': item.link.text
    } for item in soup.find_all('item')]

In [6]:
#Decoding the google news links to original source

def decode_google_url(google_url):
    """Decode Google News tracking URL to original source"""
    try:
        
        path_segments = urlparse(google_url).path.split('/')
        article_id = path_segments[-1] if path_segments else ""
        
        # Use the decoder package
        result = gnewsdecoder(
            google_url,
            interval=2,  # 2s delay between requests
            proxy=None  # Add proxy if getting blocked
        )
        
        if result['status']:
            return result['decoded_url']
        return google_url 
    
    except Exception as e:
        print(f"Decoding failed: {str(e)}")
        return google_url


In [7]:
news=get_news()
news

[{'title': 'Russell Brand charged with rape and sexual assault - BBC',
  'source': 'BBC',
  'time': 'Fri, 04 Apr 2025 15:01:05 GMT',
  'link': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE0wZmNUQV9lMlA1S0h2QnR3eGRCNUkyT25LZUpaWlh3M1pmY1pDWU55ZWJpUEdOODI3ZUVicnVkRnBRRUx5YXdDQzB5NURMOFVmZi11c09sWHhidw?oc=5'},
 {'title': 'China strikes back at Trump with own tariffs, export curbs - Reuters',
  'source': 'Reuters',
  'time': 'Fri, 04 Apr 2025 13:56:55 GMT',
  'link': 'https://news.google.com/rss/articles/CBMikAFBVV95cUxQQmVXM3NiTHJMb01GNGg4ZDhqdk52UjU3a2Fib1ZlUkF3SDNEcXZ1azQ5REZTSF9hTHVOZDBWSTl5LUstQWtINTU0dDVzVGVOaDVzenFkME5BanNkdEVtNGJzalE5ZVY4d1dOY01HRHBNNlJZVWJjMWp1U1p4ek5NVkI1OF9IZmdvaXBHcVc0OWw?oc=5'},
 {'title': 'National Security Agency chief ousted after far-right activist urged his removal - The Washington Post',
  'source': 'The Washington Post',
  'time': 'Fri, 04 Apr 2025 11:09:15 GMT',
  'link': 'https://news.google.com/rss/articles/CBMikwFBVV95cUxNekFjX3Z5Y1NucGY2Zno5

In [8]:
for item in news:
    item['link'] = decode_google_url(item['link'])
    print(item['link'])

https://www.bbc.com/news/articles/c0457d02e9go
https://www.reuters.com/world/china-impose-tariffs-34-all-us-goods-april-10-2025-04-04/
https://www.washingtonpost.com/national-security/2025/04/03/nsa-director-fired-tim-haugh/
https://www.washingtonpost.com/world/2025/04/04/india-trump-tariffs-modi-trade/
https://www.nbcnews.com/news/education/education-department-dei-discrimination-trump-funding-cuts-rcna199575
https://abcnews.go.com/Health/rfk-jr-hhs-layoffs-needed-americans-sicker-data/story?id=120450986
https://www.nytimes.com/2025/04/04/us/severe-storms-central-united-states.html
https://www.nbcnews.com/news/us-news/man-trump-accidentally-sent-el-salvador-hearing-rcna199582
https://www.cnn.com/2025/04/03/us/michigan-couple-mexico-timeshare-dispute/index.html
https://apnews.com/article/senate-budget-tax-cuts-spending-gop-trump-c43bae18e39cd7a819099376c2aee789
https://www.cnn.com/2025/04/04/asia/myanmar-earthquake-sagaing-aid-intl-hnk-dst/index.html
https://www.axios.com/2025/04/04/yo

In [9]:
news

[{'title': 'Russell Brand charged with rape and sexual assault - BBC',
  'source': 'BBC',
  'time': 'Fri, 04 Apr 2025 15:01:05 GMT',
  'link': 'https://www.bbc.com/news/articles/c0457d02e9go'},
 {'title': 'China strikes back at Trump with own tariffs, export curbs - Reuters',
  'source': 'Reuters',
  'time': 'Fri, 04 Apr 2025 13:56:55 GMT',
  'link': 'https://www.reuters.com/world/china-impose-tariffs-34-all-us-goods-april-10-2025-04-04/'},
 {'title': 'National Security Agency chief ousted after far-right activist urged his removal - The Washington Post',
  'source': 'The Washington Post',
  'time': 'Fri, 04 Apr 2025 11:09:15 GMT',
  'link': 'https://www.washingtonpost.com/national-security/2025/04/03/nsa-director-fired-tim-haugh/'},
 {'title': 'Trump’s tariffs put India and its struggling economy at a crossroads - The Washington Post',
  'source': 'The Washington Post',
  'time': 'Fri, 04 Apr 2025 10:32:31 GMT',
  'link': 'https://www.washingtonpost.com/world/2025/04/04/india-trump-ta

In [10]:
import requests
from newspaper import Article
from fake_useragent import UserAgent
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import time

# Configure session with retry logic
session = requests.Session()
retries = Retry(
    total=3,
    backoff_factor=0.5,
    status_forcelist=[500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount('https://', HTTPAdapter(max_retries=retries))

def get_full_article_content(article_url):
    """Enhanced article downloader with timeout handling"""
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://news.google.com/',
        'DNT': '1'
    }
    
    try:
        response = session.get(article_url, headers=headers, timeout=(3.05, 10))
        response.raise_for_status()

        article = Article(article_url)
        article.download(input_html=response.text)
        article.parse()
        
        return {
            'title': article.title,
            'publish_date': article.publish_date,
            'text': article.text,
            'top_image': article.top_image,
            'source_url': article_url,
        }

    except requests.exceptions.Timeout:
        print(f"⏰ Timeout skipped: {article_url}")
        return None
    except Exception as e:
        print(f"⚠️ Error processing {article_url}: {str(e)}")
        return None

def process_news_items(news_items, delay=1):
    results = []
    for idx, item in enumerate(news_items, 1):
        print(f"Processing article {idx}/{len(news_items)}...")
        article = get_full_article_content(item['link'])
        if article:
            results.append(article)
        time.sleep(delay)
    return results


newslist = process_news_items(news)
newslist

Processing article 1/38...
Processing article 2/38...
Processing article 3/38...
Processing article 4/38...
Processing article 5/38...
Processing article 6/38...
Processing article 7/38...
Processing article 8/38...
Processing article 9/38...
Processing article 10/38...
Processing article 11/38...
Processing article 12/38...
⚠️ Error processing https://www.axios.com/2025/04/04/yoon-suk-yeol-south-korea-president-impeachment-verdict: 403 Client Error: Forbidden for url: https://www.axios.com/2025/04/04/yoon-suk-yeol-south-korea-president-impeachment-verdict
Processing article 13/38...
Processing article 14/38...
⚠️ Error processing https://www.timesofisrael.com/idf-widens-ground-operation-in-northern-gaza-kills-senior-hamas-official-in-lebanon/: 403 Client Error: Forbidden for url: https://www.timesofisrael.com/idf-widens-ground-operation-in-northern-gaza-kills-senior-hamas-official-in-lebanon/
Processing article 15/38...
Processing article 16/38...
Processing article 17/38...
Processin

[{'title': 'Russell Brand charged with rape and sexual assault',
  'publish_date': None,
  'text': 'Russell Brand charged with rape and sexual assault\n\n22 minutes ago Share Save Noor Nanji • @NoorNanji Culture reporter Share Save\n\nShutterstock\n\nRussell Brand has been charged with rape, indecent assault and sexual assault between 1999 and 2005. The charges relate to four separate women. Brand has been interviewed multiple times by police since an investigation by the Sunday Times, the Times and Channel 4\'s Dispatches in September 2023 revealed multiple serious allegations against him. The 49-year-old has previously denied the allegations, calling them "very, very hurtful", and saying his relationships have been "always consensual".\n\nIn a short statement, the Metropolitan Police said it had written to Brand to inform him that he was being charged with one allegation of rape, one allegation of indecent assault, one of oral rape and two further counts of sexual assault. The force 

In [11]:
df = pd.DataFrame(newslist)

In [12]:
df

,title,publish_date,text,top_image,source_url
0,Russell Brand charged with rape and sexual ass...,None,Russell Brand charged with rape and sexual ass...,https://ichef.bbci.co.uk/news/1024/branded_new...,https://www.bbc.com/news/articles/c0457d02e9go
1,"China strikes back at Trump with own tariffs, ...",2025-04-04 00:00:00,A drone view shows containers at the terminals...,https://www.reuters.com/resizer/v2/5VFZTKK2DZL...,https://www.reuters.com/world/china-impose-tar...
2,National Security Agency chief ousted after fa...,2025-04-03 00:00:00,"The director of the National Security Agency, ...",https://www.washingtonpost.com/wp-apps/imrs.ph...,https://www.washingtonpost.com/national-securi...
3,Trump’s tariffs put India and its struggling e...,2025-04-04 00:00:00,NEW DELHI — As President Donald Trump was swor...,https://www.washingtonpost.com/wp-apps/imrs.ph...,https://www.washingtonpost.com/world/2025/04/0...
4,Education department threatens to cut public s...,2025-04-03 22:59:22.760000+00:00,The Trump administration on Thursday threatene...,https://media-cldnry.s-nbcnews.com/image/uploa...,https://www.nbcnews.com/news/education/educati...
5,RFK Jr. said HHS layoffs are needed as 'Americ...,None,"About 10,000 employees were laid off from HHS ...",https://i.abcnewsfe.com/a/110592d3-0bb7-4f75-a...,https://abcnews.go.com/Health/rfk-jr-hhs-layof...
6,Storm Stalled Over Central U.S. Brings Another...,2025-04-04 00:00:00,The two other people who died in the storms in...,https://static01.nyt.com/images/2025/04/04/mul...,https://www.nytimes.com/2025/04/04/us/severe-s...
7,Hearing set for a man Trump administration acc...,2025-04-04 14:15:40.570000+00:00,A federal judge is scheduled to hold a hearing...,https://media-cldnry.s-nbcnews.com/image/uploa...,https://www.nbcnews.com/news/us-news/man-trump...
8,Michigan couple released after nearly a month ...,2025-04-03 00:00:00,CNN —\n\nA Michigan couple was released Thursd...,https://media.cnn.com/api/v1/images/stellar/pr...,https://www.cnn.com/2025/04/03/us/michigan-cou...
9,Republicans moving ahead with Trump’s ‘big’ bi...,2025-04-03 23:28:40,"WASHINGTON (AP) — After a long wait, the Senat...",https://dims.apnews.com/dims4/default/a8c32ca/...,https://apnews.com/article/senate-budget-tax-c...


# Time to do ai analysis

In [13]:

import numpy as np # linear algebra
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import nltk
from nltk.corpus import stopwords

# 1. Create and fit a temporary vectorizer with sample vocabulary
sample_corpus = [
    "fake news political scandal fraud cryptocurrency",
    "real genuine authentic trustworthy verified"
]
vectorizer = CountVectorizer(max_features=1000)
vectorizer.fit(sample_corpus) 

# 2. Model setup with original architecture
model = Sequential([
    Dense(100, activation='relu', input_dim=1023669),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 3. Load weights manually
weights = np.load(r'C:\Users\Ashwin\Desktop\MAD_LAB_PROJECT\NewsScraper\model_weights.npy', allow_pickle=True)
for layer, (weights, bias) in zip(model.layers, weights.reshape(-1, 2)):
    layer.set_weights([weights, bias])
    
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))


c:\Users\Ashwin\Desktop\MAD_LAB_PROJECT\NewsScraper\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:

def preprocess(text):
    """Basic text cleaning"""
    return ' '.join([
        word.lower() 
        for word in text.split() 
        if word.lower() not in stop_words and word.isalpha()
    ])
def analyze_article(article, top_n=5):
    """
    Predict whether an article is Fake or Real, with confidence and key words.

    Args:
    article (str): The text of the article to analyze.
    top_n (int): Number of top influential words to return.

    Returns:
    dict: Containing prediction, confidence, and top influential words.
    """
    # Preprocess the article
    processed = preprocess(article)

    # Transform the article into a BOW representation
    bow_vector = vectorizer.transform([processed]).toarray()
    # Pad the BOW vector to match the model's input shape (1023669 features)
    padded_bow = np.zeros((1, 1023669))  # Create a zero array with required shape
    padded_bow[0, :bow_vector.shape[1]] = bow_vector  # Copy existing features into padded array

    # Get prediction confidence
    confidence = model.predict(padded_bow, verbose=0)[0][0]
    prediction = 'Fake' if confidence > 0.5 else 'Real'

    # Get feature importance using first layer weights
    input_weights = model.layers[0].get_weights()[0]  # First layer weights
    feature_importance = padded_bow.dot(input_weights)  # Multiply BOW by weights

    # Extract only words present in the article (non-zero BOW entries)
    feature_names = vectorizer.get_feature_names_out()
    non_zero_indices = np.nonzero(bow_vector[0])[0]  # Indices of non-zero features
    keywords_with_scores = [(feature_names[i], feature_importance[0, i]) for i in non_zero_indices]

    # Sort by absolute impact score and select top N keywords
    sorted_keywords = sorted(keywords_with_scores, key=lambda x: abs(x[1]), reverse=True)[:top_n]
    return {
        'prediction': prediction,
        'confidence': float(confidence if prediction == 'Fake' else 1 - confidence),
        'keywords': sorted_keywords
    }

# Summariser

In [15]:
import nltk 
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ashwin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ashwin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Ashwin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [16]:
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist

def text_summarizer(text, num_sentences=3):
    # Text into sentences
    sentences = sent_tokenize(text)

    # Text into words
    words = word_tokenize(text.lower())

    # Removing stop words
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.casefold() not in stop_words]

    # Calculate word frequencies
    fdist = FreqDist(filtered_words)

    # Assign scores to sentences based on word frequencies
    sentence_scores = [sum(fdist[word] for word in word_tokenize(sentence.lower()) if word in fdist)
                       for sentence in sentences]

    # Create a list of tuples containing sentence index and score
    sentence_scores = list(enumerate(sentence_scores))

    # Sort sentences by scores in descending order
    sorted_sentences = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

    # Randomly select the top `num_sentences` sentences for the summary
    random_sentences = random.sample(sorted_sentences, num_sentences)

    # Sort the randomly selected sentences based on their original order in the text
    summary_sentences = sorted(random_sentences, key=lambda x: x[0])

    # Create the summary
    summary = ' '.join([sentences[i] for i, _ in summary_sentences])

    return summary

# Firebase uploader

In [17]:
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase
cred = credentials.Certificate(r"C:\Users\Ashwin\Desktop\MAD_LAB_PROJECT\NewsScraper\truthlens-68bed-firebase-adminsdk-fbsvc-4c3626b3ec.json")  #Get your own firebase credentials and put here
firebase_admin.initialize_app(cred)

In [18]:


# Get Firestore database
db = firestore.client()

def upload_article(title, article, confidence, image, source, tags):
    """
    Uploads an article to the Firestore collection 'articles'.
    
    Args:
        title (str): Title of the article.
        article (str): Full text of the article.
        confidence (float): Confidence score of the article.
        image (str): URL of the article image.
        source (str): Source URL of the article.
        tags (list): List of tags related to the article.

    Returns:
        str: Document ID of the uploaded article.
    """
    data = {
        "title": title,
        "article": article,
        "confidence": confidence,
        "image": image,
        "source": source,
        "tags": tags
    }
    
    doc_ref = db.collection("news").add(data)
    return doc_ref[1].id  # Returns the auto-generated document ID


# Final output

In [20]:
for i in newslist:
    topimage=i['top_image']
    asource=i['source_url']
    atitle=i['title']
    text=i['text']
    summary=text_summarizer(text)
    analysis = analyze_article(text)
    confidence_rating = analysis['confidence']
    confidence_rating=round(confidence_rating * 100, 1)
    keywords = analysis['keywords']
    words=[]
    
    for w,s in keywords:
        words.append(w)
    print("Title: ", atitle)
    print("Confidence: ", confidence_rating)
    print("Keywords: ", keywords)

Title:  Russell Brand charged with rape and sexual assault
Confidence:  55.8
Keywords:  []
Title:  China strikes back at Trump with own tariffs, export curbs
Confidence:  55.8
Keywords:  []
Title:  National Security Agency chief ousted after far-right activist urged his removal
Confidence:  55.8
Keywords:  []
Title:  Trump’s tariffs put India and its struggling economy at a crossroads
Confidence:  59.9
Keywords:  [('news', np.float64(0.001031254418194294))]
Title:  Education department threatens to cut public school funding over DEI practices
Confidence:  55.8
Keywords:  []
Title:  RFK Jr. said HHS layoffs are needed as 'Americans are getting sicker.' Here's what the data shows.
Confidence:  71.2
Keywords:  [('news', np.float64(0.004125017672777176))]
Title:  Storm Stalled Over Central U.S. Brings Another Day of Rain and Flooding
Confidence:  55.8
Keywords:  []
Title:  Hearing set for a man Trump administration accidentally sent to notorious El Salvador jail
Confidence:  63.7
Keywords: